<h1><center> Aula 11 - Floresta Aleatória</center></h1>

Na aula de prática de hoje iremos aprender a usar o método de aprendizagem de máquina chamado Floresta Aleatória (<i>Random Forest</i>). Floresta aleatórias é um dos algoritmos mais utilizados, devido à sua simplicidade e o fato de que pode ser utilizado para tarefas de classificação e também de regressão, que serão abordadas a seguir. 
<br><br>
Iremos utilizar novamente a <i>dataset</i> do <a href="https://drive.google.com/open?id=1bvQeIl0EUDW_UVTARilsTW3PLDuuhlxl"><b>Titanic</b></a> (em português) (<a href = "https://www.kaggle.com/c/titanic/data">dataset original - Kaggle</a>) . Primeiramente iremos fazer as devidas transformações nos dados, como seleção de <i>features</i>, remoção de valores nulos, e <i>"dummização"</i>. Depois, iremos separar o <i>dataset</i> em treino e teste. Em seguida, iremos os conceitos de Grid Search e Cross-validation para selecionar os melhores parametros para criar nossa floresta. Por fim, apresentaremos os resultados do nosso modelo e salvaremos o mesmo em um arquivo, para que possamos carregado-lo e utilizar a predição novamente, sem precisarmos criar o modelo do zero novamente.
<br><br>
Documentação <a href = https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html> Random Forest </a>

<h1><b>LET'S CODE!!</b></h1>

In [0]:
# BIBLIOTECAS DATA SCIENCE
import pandas as pd
import numpy as np

# BIBLIOTECAS DE PLOT
import matplotlib.pyplot as plt 

In [0]:
# FUNÇÕES UTEIS
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    plt.grid(False)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)    

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    # DEIXAR QUADRADO
    plt.ylim(len(cm)-0.5, -0.5)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# MOSTRA RESULTADOS
def mostra_resultados(y, y_pred, classes, normalize=False,
                      title='Confusion matrix', cmap=plt.cm.coolwarm):

    # Calcular os resultados
    cm = confusion_matrix(y, y_pred)
    FP = cm.sum(axis=0) - np.diag(cm)  # Falso Positivos
    FN = cm.sum(axis=1) - np.diag(cm)  # Falso Negativos
    TP = np.diag(cm)                   # Verdadeiro Positivo
    TN = cm.sum() - (FP + FN + TP)     # Verdadeiro Negativos
    TPR = TP / (TP + FN)  # Revocação    
    PPV = TP / (TP + FP)  # Precisao
    F1 = 2 * ((PPV * TPR) / (PPV + TPR)) # F1 Score
    ACC = (TP + TN) / (TP + FP + FN + TN)  # Acurácia

    # Exibe resultados
    print("    Verdadeiros Positivos:{}".format(TP))
    print("    Verdadeiros Negativos:{}".format(TN))
    print("    Falso Positivo:{}".format(FP))
    print("    Falso Negativo:{}".format(FN))
    
    print("    Revocação:{}".format(TPR))    
    print("    Precisao:{}".format(PPV))
    
    print("    F1 Score:{}".format(F1))
    print("    Acuracia:{}".format(ACC))
    
    plot_confusion_matrix(cm, classes)    

In [0]:
# CARREGA BASE DO TITANIC
# ----------------------------------------------------------------------------------------------------- #
# OBS.: O TRECHO DE CÓDIGO ABAIXO SÓ É NECESSÁRIO REALIZAR SE ESTIVER USANDO O AMBIENTE DO GOOGLE COLAB
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
arquivo_base = "/content/drive/My Drive/titanic_ptbr.csv"
# ----------------------------------------------------------------------------------------------------- #

# arquivo_base = <caminho do arquivo>

# URL KAGGLE DA BASE: https://www.kaggle.com/c/titanic

# CARREGAR A BASE NO DATAFRAME
df = pd.read_csv(arquivo_base, sep=";", encoding="utf8")

**CAMPOS (FEATURES)**
* **id_passageiro** = ID do passageiro do navio (código primário).
* **sobreviveu** = Se sobreviveu ao naufrágio estará como 1 e caso esteja com 0 (zero não sobreviveu.
* **tipo_classe** = Tipo de classe de passagem (Do 1 ao 3), sendo 1 a melhor classe e 3 a pior classe.
* **nome** = Nome do passageiro
* **sexo** = Gênero do passageiro, sendo masculino e feminino.
* **idade** = Idade do passageiro na data da ocorrência do naufrágio.
* **n_irmaos_abordo** = Número de irmãos / cônjuges a bordo.
* **n_pais_filhos_abordo** = Número de pais / filhos a bordo.
* **ticket** = Código do ticket.
* **preco** = Valor da passagem.
* **cabine** = Código de identificação da Cabine.
* **local_embarque** = Local onde o passageiro embarcou no navio.

In [0]:
# CRIANDO UM MODELO RANDOM FOREST CLASSIFIER
from sklearn.ensemble import RandomForestClassifier

modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

In [0]:
# VERIFICANDO QUALIDADE DO MODELO
y_pred = modelo.predict(X_test)
y_pred

In [0]:
mostra_resultados(y_test, y_pred, modelo.classes_)

In [0]:
# PARAMETROS DA GRID
parametros_grid = {
    "n_estimators": [10, 50, 100, 200, 400],
    "max_depth": [5, 10, 20, 40],
    "max_features": ["auto"],
    "bootstrap": [True],
    "n_jobs": [-1]
}

n_cross_validation = 5
verbose = 10

In [0]:
# PROCESSAMENTO DO GRID SEARCH
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

modelo_floresta_aleatoria = GridSearchCV(RandomForestClassifier(), scoring="accuracy",
                                         param_grid=parametros_grid, cv=n_cross_validation, verbose=verbose)

In [0]:
# FITTING E CROSS-VALIDATION
modelo_floresta_aleatoria.fit(X, y)

In [0]:
# MELHOR PARAMENTRO
melhor_modelo_floresta_aleatoria = modelo_floresta_aleatoria.best_estimator_ 
modelo_floresta_aleatoria.best_params_, modelo_floresta_aleatoria.best_score_

In [0]:
features = dummy.columns

# IMPORTÂNCIA DE CADA FEATURE
importancia_features = zip(melhor_modelo_floresta_aleatoria.feature_importances_, features)
for importancia, feature in sorted(importancia_features, reverse=True):
    print("%s: %f%%" % (feature, importancia*100))

Em alguns casos, para melhorar o desempenho do modelo, será necessário realizar um balanceamento dos possíveis valores da <i>feature</i> resposta (<i>target</i>). Não é o caso do <i>dataset</i> do <b>Titanic</b>, mas iremos comparar os resultados dos modelos com balanceamento e sem posteriormente.

Para isso, iremos utilizar a função <a href="https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html"><b>resample</b></a>, do <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.utils">sklearn.utils</a>.

In [0]:
# CRIANDO UMA DUMMY BALANCEADA 50/50 COM RESAMPLE (sklearn.utils)
from sklearn.utils import resample

sobreviveu_dummy = dummy[dummy["sobreviveu"] == 1]
nao_sobreviveu_dummy = dummy[dummy["sobreviveu"] == 0]

# COMO A QUANTIDADE DE SOBREVIVENTES É MENOR QUE A QUANTIDADE DE NÃO SOBREVIVENTES, 
# IREMOS USAR O TAMANHO DA DUMMY DE SOBREVIVENTES COMO REFERÊNCIA PARA REAMOSTRAGEM

# PARAMETROS DO RESAMPLE
# n_samples = número de amostras aleatórias
# random_state = seed que será utilizada no processo de extração aleatória
tamanho_amostra = sobreviveu_dummy.shape[0]

nao_sobreviveu_dummy_amostra = resample(nao_sobreviveu_dummy, n_samples=tamanho_amostra)

dummy_balanceada = pd.concat([sobreviveu_dummy, nao_sobreviveu_dummy_amostra])

In [0]:
# VERIFICA QUANTIDADE DE SOBREVIVENTES E NÃO SOBREVIVENTES (PERCENTUAL)
round(dummy_balanceada["sobreviveu"].value_counts(normalize=True) * 100, 2)

In [0]:
# ATRIBUINDO FEATURE RESPOSTA (TARGET/LABEL) EM y
y = dummy_balanceada["sobreviveu"]

In [0]:
# ATRIBUINDO FEATURES A X (SEM TARGET/LABEL)
X = dummy_balanceada.drop(columns="sobreviveu")

In [0]:
# SEPARANDO TREINO E TESTE
from sklearn.model_selection import train_test_split

test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [0]:
# CRIANDO UM MODELO RANDOM FOREST CLASSIFIER
from sklearn.ensemble import RandomForestClassifier

modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

In [0]:
# VERIFICANDO QUALIDADE DO MODELO
y_pred = modelo.predict(X_test)
y_pred

In [0]:
mostra_resultados(y_test, y_pred, modelo.classes_)